##  SETUP VIRTUAL MACHINES FOR MULTIPLE USERS
Use Azure Virtual Machine Scale Set (VMSS) and Azure Command Line Interface (Az CLI) to create N virtual machines. After setting up we invoke a setup script on each VM to setup the environment. 



## Azure Resource Setup

In [ ]:
AZURE_SUBSCRIPTION_ID = "your-subscription-id"
LOCATION = "westus"  # e.g. westus
VM_SIZE = "Standard_D8_v3"  # e.g. Standard_D8_v3
RG_NAME = "mladsfall2019"
VMSS_NAME = "{}-vmss".format(RG_NAME)
ADMIN_USER_NAME = "vmadmin"
ADMIN_PASSWORD = "nlpbpMlads2019"

# Number of VM instances in a scale set 
NUM_VM = 10

In [ ]:
!az login

In [ ]:
!az account set -s {AZURE_SUBSCRIPTION_ID}

In [ ]:
!az account show

### Create VMSS

In [ ]:
!az group create --name {RG_NAME} --location {LOCATION}

In [ ]:
# Setup public-ip for each vm by using `--public-ip-per-vm` parameter
!az vmss create -g {RG_NAME} -n {VMSS_NAME} --instance-count {NUM_VM} --image microsoft-dsvm:linux-data-science-vm-ubuntu:linuxdsvmubuntu:latest --vm-sku {VM_SIZE} --public-ip-per-vm --admin-username {ADMIN_USER_NAME} --admin-password {ADMIN_PASSWORD}

In [ ]:
# Check public ip addresses of the instances
!az vmss list-instance-public-ips --resource-group {RG_NAME} --name {VMSS_NAME} | grep 'ipAddress'

## Create User Accounts and Environment Setup

In [ ]:
SCRIPT = "setup_vm.sh"

In [ ]:
!az vmss list-instances -n {VMSS_NAME} -g {RG_NAME} --query "[].id" --output tsv | az vmss run-command invoke --command-id RunShellScript --scripts @{SCRIPT} --ids @-